## IMPORTING LIBRARIES 

In [1]:
import os
import glob
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM
from keras.layers import TimeDistributed 
from pydub import AudioSegment
import sounddevice as sd
from PIL import Image, ImageTk 

C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## DEFINING FEATURE EXTRACTION METHOD

In [2]:
def extract_features(file_path):
    """
    Extracts comprehensive audio features with dynamic n_fft adjustment for short clips.
    """
    audio, sr = librosa.load(file_path, res_type='scipy')
    # Trim silence
    audio, _ = librosa.effects.trim(audio, top_db=20)
    # Adjust n_fft dynamically based on audio length
    n_fft_value = min(1024, len(audio))
    
    # Extract features
    mfcc = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sr, n_fft=n_fft_value).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft_value).T, axis=0)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sr, n_fft=n_fft_value).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sr).T, axis=0)
    rms = np.mean(librosa.feature.rms(y=audio).T, axis=0)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)
    
    # Combine all features into a single array
    features = np.concatenate([mfcc, chroma, mel, spectral_contrast, tonnetz, rms, zcr])
     # Normalize the features
    features_normalized = (features - np.mean(features)) / np.std(features)
    
    return features_normalized
    


## LOADING TESS DATASET

In [ ]:

base_path = r'D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data'

data = []
labels = []
for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)  # Construct folder path
    # Check if the path is a directory
    if os.path.isdir(folder_path):
        # Find all .wav files in the folder
        for file_path in glob.glob(os.path.join(folder_path, '*.wav')):
            print(file_path)  # Print the file path for debugging/monitoring
            features = extract_features(file_path)  # Extract features (assumes extract_features is defined)
            data.append(features)  # Append features to the data list
            labels.append(folder_name) 

D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_angry\OAF_back_angry.wav


C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=816
  warnings.warn(
C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=408
  warnings.warn(


D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_angry\OAF_bar_angry.wav


C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=944
  warnings.warn(
C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=472
  warnings.warn(


D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_angry\OAF_base_angry.wav


...

D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_happy\OAF_bar_happy.wav
D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_happy\OAF_base_happy.wav


C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=584
  warnings.warn(


D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_happy\OAF_bath_happy.wav
D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_happy\OAF_bean_happy.wav


C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=560
  warnings.warn(


D:\kagglehub\datasets\ejlok1\toronto-emotional-speech-set-tess\versions\1\TESS Toronto emotional speech set data\OAF_happy\OAF_beg_happy.wav


## CREATING LSTM MODEL

In [4]:
# Ensure that the labels are correctly encoded as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Reshaping input data for LSTM (ensure the shape is (samples, time_steps, features))
X_train = np.array(X_train)[:, np.newaxis, :]
X_test = np.array(X_test)[:, np.newaxis, :]

# Define the model
model = Sequential()
model.add(TimeDistributed(Dense(256, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy',])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50
70/70 [==============================] - 5s 21ms/step - loss: 2.5993 - accuracy: 0.1174 - val_loss: 2.3901 - val_accuracy: 0.4482
Epoch 2/50
70/70 [==============================] - 0s 5ms/step - loss: 2.0795 - accuracy: 0.3004 - val_loss: 1.5551 - val_accuracy: 0.4964
Epoch 3/50
70/70 [==============================] - 0s 5ms/step - loss: 1.4638 - accuracy: 0.4821 - val_loss: 1.0336 - val_accuracy: 0.7500
Epoch 4/50
70/70 [==============================] - 0s 5ms/step - loss: 1.1184 - accuracy: 0.5942 - val_loss: 0.7762 - val_accuracy: 0.7929
Epoch 5/50
70/70 [==============================] - 0s 5ms/step - loss: 0.9074 - accuracy: 0.6795 - val_loss: 0.6146 - val_accuracy: 0.8429
Epoch 6/50
70/70 [==============================] - 0s 6ms/step - loss: 0.7665 - accuracy: 0.7286 - val_loss: 0.5202 - val_accuracy: 0.8411
Epoch 7/50
70/70 [==============================] - 0s 6ms/step - loss: 0.6844 - accuracy: 0.7621 - val_loss: 0.4381 - val_accuracy: 0.9018
Epoch 8/50
70/70 [=

## MODEL SUMMARY

In [5]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 1, 256)            43264     
 ributed)                                                        
                                                                 
 dropout (Dropout)           (None, 1, 256)            0         
                                                                 
 lstm (LSTM)                 (None, 128)               197120    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 14)                1806      
                                                                 
Total params: 242190 (946.05 KB)
Trainable params: 242190 (946.05 KB)
Non-trainable params: 0 (0.00 Byte)
________________

## EVALUATING MODEL PERFORMANCE

In [6]:

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

y_pred = np.argmax(model.predict(X_test), axis=-1)  # Get predicted class labels

# Calculate Precision and Recall manually
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

# Print Precision and Recall
print(f"Precision = {precision:.4f}")
print(f"Recall = {recall:.4f}")



18/18 [==============================] - 0s 2ms/step - loss: 0.1195 - accuracy: 0.9571
Test Loss: 0.11948093771934509
Test Accuracy: 0.9571428298950195
18/18 [==============================] - 1s 3ms/step
Precision = 0.9591
Recall = 0.9584


## CREATING EMOTION PREDICTION FUNCTION

In [7]:
def predict_emotion(audio_file):
    features = extract_features(audio_file)
    features = features[np.newaxis, np.newaxis, :]  
    print("Features shape:", features.shape)
    print("Features:", features)

    predicted_probabilities = model.predict(features)
    print("Predicted probabilities shape:", predicted_probabilities.shape)
    print("Predicted probabilities:", predicted_probabilities)

    predicted_label_index = np.argmax(predicted_probabilities)
    print("Predicted label index:", predicted_label_index)

    predicted_emotion = label_encoder.classes_[predicted_label_index]
    print("Predicted emotion:", predicted_emotion)

    # Emotion mapping for TESS dataset
    
    emotion_mapping = {
        'YAF_angry': 'ANGRY',
        'YAF_disgust': 'DISGUST',
        'YAF_fear': 'FEAR',
        'YAF_happy': 'HAPPY',
        'YAF_neutral': 'NEUTRAL',
        'YAF_pleasant_surprised': 'SURPRISED',
        'YAF_sad': 'SAD',
        'OAF_angry': 'ANGRY',
        'OAF_disgust': 'DISGUST',
        'OAF_Fear': 'FEAR',
        'OAF_happy': 'HAPPY',
        'OAF_neutral': 'NEUTRAL',
        'OAF_Pleasant_surprised': 'SURPRISED',
        'OAF_Sad': 'SAD',
    }


    recognizable_emotion = emotion_mapping.get(predicted_emotion)
    return recognizable_emotion


## SAVING THE MODEL

In [7]:
model.save("ERM.h5")

C:\Users\NEW\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## THANK YOU